# TrES-3b transmission spectroscopy: contamination

*<a href="mailto:hannu.parviainen@physics.ox.ac.uk">Hannu Parviainen</a>, University of Oxford*

***The aperture used to calculate the TrES-3 spectra contains a faint background star, here we calculate how much it contributes to the total flux.***

This notebook is a part in a series of notebooks working as an appendix to Parviainen et al. (2015, submitted), aiming to make the analysis presented in the paper easily reproductable. The results and the figures presented in the paper can be reproduced by following these notebooks in the numbered order.

*Dependencies:* See `000_README.ipynb` for the list of required Python packages, and how to install them.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from glob import glob
from scipy.ndimage import median_filter as mf
from scipy.optimize import fmin

from pycntr import psf_g1d
from core import *

datadir = '../../../data/GTC9-14A/OB0008/'

In [3]:
def multig_model(pv, size, ncomp=3, return_comps=False):
    m = zeros([ncomp, size])
    for i in range(ncomp):
        m[i,:] = psf_g1d(pv[0], pv[1+i*2], pv[2+i*2], size)
    
    if return_comps:
        return m
    else:
        return m.sum(0)
        
def model(pv, size, nct=3, ncc=2, return_comps=False):
    m_target = multig_model(pv[:1+2*nct], size, nct)
    m_contam = multig_model(pv[1+2*nct:], size, ncc)
    
    if return_comps:
        return m_target, m_contam
    else:
        return m_target+m_contam 
    
def integrate_mg(pv):
    """Returns the integral of n Gaussians with an amplitude A and fwhm F."""
    return sum(pv[0::2]*pv[1::2]/2.355*sqrt(2*pi))

In [4]:
files = sorted(glob(join(datadir,'object','*.fits')))[3:]
gain = pf.getval(files[0], 'gain', ext=0)

IndexError: list index out of range

In [6]:
f  = pf.getdata(files[3],2) * gain
fl = f[500:700:,400:450].mean(0)
sk = f[500:700:,445:475].mean()
lfl = log10(fl-sk)

IndexError: list index out of range

In [7]:
semilogy(fl-sk)
ylim(10,3.2e4);

NameError: name 'fl' is not defined

In [8]:
res = fmin(lambda pv: ((lfl-log10(model(pv, fl.size,4,3,False)))**2).sum(), 
           [15.5, 2e4, 4.2,  5e3, 7,  1e3, 15,  1e2, 40,   
            31, 2e2,5, 5e1, 10, 5, 15], maxfun=40000, maxiter=40000)

NameError: global name 'lfl' is not defined

In [9]:
mt,mc = model(res, fl.size, 4, 3, True)

NameError: name 'res' is not defined

In [10]:
x = arange(400,400+fl.size)
with sb.plotting_context('paper', rc=rc_paper):
    with sb.axes_style('ticks'):
        fig,ax = subplots(1,1,figsize=(AAOCW,0.6*AAOCW))
        ax.plot(x,lfl, '-', lw=5, c=c_ob)
        ax.plot(x,log10(mt+mc), 'w-', lw=1.5)
        ax.plot(x,log10(mt), '--', c=c_bo, alpha=0.5, lw=1.5)
        ax.plot(x,log10(mc), '--', c=c_bo, alpha=0.5, lw=1.5)
    setp(ax, ylim=(0.5,4.6), ylabel='Log$_{10}$ flux', xlabel='Pixel')
    sb.despine(fig)
    fig.tight_layout()
    fig.savefig('plots/pub_contamination.pdf')

NameError: name 'fl' is not defined

In [11]:
I1 = integrate_mg(res[1:1+2*4])
I2 = integrate_mg(res[2+2*4:])
print I2/(I1+I2)

NameError: name 'res' is not defined

---
<center> &copy; 2015 <a href="mailto:hannu.parviainen@physics.ox.ac.uk">Hannu Parviainen</a></center>